In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Hi,\
**Welcome to my Notebook !!**.\
The following notebook deals with finding the best params with optuna study.

**Please leave an upvote or feedback, if you like or use at part of it. \
Would encourage me more to share.**


I have finally used the obtained params in this notebook:- \
https://www.kaggle.com/skiller/a-z-predictive-modelling-welcome 

I have **not doing any EDA in the current notebook** as i have already taken care of it in the above notebook. 

Warm Regards

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import warnings

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns

warnings.filterwarnings('ignore')

In [ ]:
# import datasets
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/test.csv')
submission = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv')

In [ ]:
train_df['num_nulls'] = train_df.drop(['id', 'claim'], axis = 1).isna().sum(axis = 1)
test_df['num_nulls'] = test_df.drop(['id'], axis = 1).isna().sum(axis = 1)

# Removal of null values. (Also a bit of Preprocessing) 
But we can't drop the rows owing to the large amount single null rows

In [ ]:
%%time
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer,  KBinsDiscretizer
from sklearn.impute import SimpleImputer

features = [col for col in train_df.columns if col not in ['claim', 'id']]
pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='median',missing_values=np.nan)),
        ("scaler", QuantileTransformer(n_quantiles=64,output_distribution='uniform')),
        ('bin', KBinsDiscretizer(n_bins=64, encode='ordinal',strategy='uniform'))
        ])
train_df[features] = pipe.fit_transform(train_df[features])
test_df[features] = pipe.transform(test_df[features])

# Ok now Lets find the params for our models 
3 models to be looked here are -> **XGBoost, CatBoost, LGDM**

In [ ]:
# import packages
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier 
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from tqdm import tqdm
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import optuna

# CAT Boost Best Params

In [ ]:
OPTUNA_OPTIMIZATION = True

def objective(trial):    
    params = {
        'iterations':trial.suggest_int("iterations", 1000, 20000),
        'objective': trial.suggest_categorical('objective', ['Logloss', 'CrossEntropy']),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
        'od_wait':trial.suggest_int('od_wait', 500, 2000),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.001,1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'random_strength': trial.suggest_uniform('random_strength', 10, 50),
        'depth': trial.suggest_int('depth',1 , 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 15),
#         'task_type' : 'GPU',
        'devices' : '0'
    }
    
    if params['bootstrap_type'] == 'Bayesian':
        params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 10)
    elif params['bootstrap_type'] == 'Bernoulli':
        params['subsample'] = trial.suggest_float('subsample', 0.1, 1)

    training_df, validation_df =  train_test_split(train_df, test_size=0.1, shuffle=True, random_state=1)

    model = CatBoostClassifier(**params)
    
    x_train = training_df.drop(['claim', 'id'],axis=1) 
    y_train = training_df['claim']
    x_valid = validation_df.drop(['claim', 'id'],axis=1) 
    y_valid = validation_df['claim']
    model.fit(
        x_train , y_train,
        eval_set=[(x_valid, y_valid)],
        early_stopping_rounds=100,
        use_best_model=True,
        verbose=0
    )
    
    return roc_auc_score(y_valid,  model.predict_proba(x_valid)[:,1])

In [ ]:
%%time
study = optuna.create_study(
    direction='maximize',
    study_name='CatbClf'
)

study.optimize(
    objective,
    n_trials=100
)

In [ ]:
print(f"Best Trial: {study.best_trial.value}")
print(f"Best Params: {study.best_trial.params}")

Lets look at params dependencies with accuracy

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

Delete Study from RAM

In [ ]:
del study

# XGBOOST Best Params

In [ ]:
OPTUNA_OPTIMIZATION = True

def objective(trial):    
    params = {
            'n_estimators':trial.suggest_int("n_estimators", 1000, 20000),
            'learning_rate' : trial.suggest_uniform('learning_rate', 0.001, 1),
            'subsample': trial.suggest_uniform('subsample', 0.1, 1),
            'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.1, 1),
            'max_depth': trial.suggest_categorical('max_depth', [1,3,5,7,9,11,13,15,17,20]),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 10.0),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 10.0),
#             'tree_method': 'gpu_hist'
        }
    
    model = XGBClassifier(**params)
    
    training_df, validation_df =  train_test_split(train_df, test_size=0.1, shuffle=True, random_state=1)

    x_train = training_df.drop(['claim', 'id'],axis=1) 
    y_train = training_df['claim']
    x_valid = validation_df.drop(['claim', 'id'],axis=1) 
    y_valid = validation_df['claim']

    model.fit(
        x_train , y_train,
        eval_set=[(x_valid, y_valid)],
        early_stopping_rounds=100,
        verbose=0
    )
    
    return roc_auc_score(y_valid,  model.predict_proba(x_valid)[:,1])

In [ ]:
%%time
study = optuna.create_study(
    direction='maximize',
    study_name='XG_boost'
)

study.optimize(
    objective,
    n_trials=100
)

In [ ]:
print(f"Best Trial: {study.best_trial.value}")
print(f"Best Params: {study.best_trial.params}")

Lets look at params dependencies with accuracy

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

Delete study from RAM

In [ ]:
del study

# LGDM CLASSIFIER Best Params

In [ ]:
OPTUNA_OPTIMIZATION = True

def objective(trial):    
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbose": -1,
        "boosting_type": "gbdt",
        'n_estimators':trial.suggest_int("n_estimators", 1000, 20000),
        'learning_rate' : trial.suggest_uniform('learning_rate', 0.001, 1),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-5, 10.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-5, 10.0),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.1, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.1, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 300),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 300),
        'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.1, 1),
#         "device_type" : "gpu"
    }

    training_df, validation_df =  train_test_split(train_df, test_size=0.1, shuffle=True, random_state=1)

    model = LGBMClassifier(**params)
    
    x_train = training_df.drop(['claim', 'id'],axis=1) 
    y_train = training_df['claim']
    x_valid = validation_df.drop(['claim', 'id'],axis=1) 
    y_valid = validation_df['claim']
    model.fit(
        x_train , y_train,
        eval_set=[(x_valid, y_valid)],
        early_stopping_rounds=100,
        verbose=-1,
    )
    
    return roc_auc_score(y_valid,  model.predict_proba(x_valid)[:,1])

In [ ]:
%%time
study = optuna.create_study(
    direction='maximize',
    study_name='LGDM'
)

study.optimize(
    objective,
    n_trials=100
)

In [ ]:
print(f"Best Trial: {study.best_trial.value}")
print(f"Best Params: {study.best_trial.params}")

Lets look at params dependencies with accuracy

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

Delete Study

In [ ]:
del study